In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import pymongo
import pandas as pd
import requests

In [3]:
executable_path = {'executable_path':'/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)
browser = Browser("chrome", **executable_path, headless = True)

In [4]:
url = ('https://mars.nasa.gov/news/')

response = requests.get(url)
soup = bs(response.text, 'html.parser')
type(soup)
#print(response.text)

bs4.BeautifulSoup

In [5]:
# Extract the text from the class="content_title" and clean up the text use strip
news_title = soup.find_all('div', class_='content_title')[0].find('a').text.strip()

#print title to check
print(news_title)

NASA Invites Students to Name Mars 2020 Rover


In [6]:
# Extract the paragraph from the class="rollover_description_inner" and clean up the text use strip
news_paragraph = soup.find_all('div', class_='rollover_description_inner')[0].text.strip()

#print paragraph to check
print(news_paragraph)

Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.


In [8]:
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_url)
#img = browser.find_by_id("full_image")
#img.click()

In [11]:
html = browser.html
soup = bs(html, "html.parser")
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23479-640x350.jpg


In [13]:
# Scrape the latest tweet
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
twitter_html = browser.html
twitter_soup = bs(twitter_html, 'html.parser')

tweets = twitter_soup.find('ol', class_='stream-items')
mars_weather = tweets.find('p', class_="tweet-text").text
print(mars_weather)

InSight sol 288 (2019-09-18) low -102.3ºC (-152.1ºF) high -26.3ºC (-15.3ºF)
winds from the SW at 4.9 m/s (11.0 mph) gusting to 17.7 m/s (39.7 mph)
pressure at 7.40 hPapic.twitter.com/Dg8txhSdeL


In [14]:
# Scrape facts
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)
mars_data = pd.read_html(facts_url)
mars_data = pd.DataFrame(mars_data[0])
mars_facts = mars_data.to_html(header = False, index = False)
print(mars_facts)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <td>Temperature:</td>
      <td>-153 to 20 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


In [17]:
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(hemispheres_url)
html = browser.html
soup = bs(html, "html.parser")
mars_hemisphere = []

result = soup.find("div", class_ = "result-list" )
hemispheres = result.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup= bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})
    
mars_hemisphere

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]